# Calculating the reserve Factors of the panels 

# Please enter your name 

In [1]:
name = "yannis"

# Imports 

In [2]:
import pandas as pd
import sys 
import os
sys.path.insert(0, os.path.abspath('..'))
sys.path.insert(0, os.path.abspath('../formulas'))
sys.path.insert(0, os.path.abspath('../optimization'))

from formulas.panels import *
from formulas.helpers import *
from formulas.abd_matrix import *

 

In [3]:
'''# config parser
import configparser

# get the rounding_digits from the ini file
config = configparser.ConfigParser()
config.read('../config.ini')
rounding_digits = int(config['DEFAULT']['rounding_digits'])'''

"# config parser\nimport configparser\n\n# get the rounding_digits from the ini file\nconfig = configparser.ConfigParser()\nconfig.read('../config.ini')\nrounding_digits = int(config['DEFAULT']['rounding_digits'])"

# Constants (Changed based on your data)

In [4]:
personal_data = personal_data_provider(name)
E_11 = personal_data[0]
E_22 = personal_data[1]
G_12 = personal_data[2]
nu_12 = personal_data[3]
knockdown = 0.9 
print(f"Your personal data is: E11 = {E_11}, E22 = {E_22}, G12 = {G_12}, nu12 = {nu_12}. Please verify.")

# Panel laminate properties 
panelStack = [45,45,-45,-45,0,0,90,90,90,90,0,0,-45,-45,45,45]
tPanel = 0.552 

#Panel constants 
length = 750
width = 400
panelthickness = 8.832


Your personal data is: E11 = 132583.92, E22 = 10198.76, G12 = 5099.38, nu12 = 0.33. Please verify.


# Import needed ABD values 

In [5]:
ABD_panel, ABD_panel_inverse = calculateABD(stacksequence=panelStack, plyT=tPanel, EModulus1=E_11, EModulus2=E_22, ShearModulus=G_12)
ABD_panel = ABD_panel * knockdown
ABD_panel_inverse = ABD_panel_inverse * knockdown

## Extract ABD parameters 


In [6]:
D11 = ABD_panel[3][3]
D22 = ABD_panel[4][4]
D12 = ABD_panel[3][4]
D66 = ABD_panel[5][5]
print(D11, D22, D12, D66)

2698671.142592391 2100821.66410216 1495574.8566103033 1583690.6878573217


# Open stress values & properties of panels 

In [7]:
# New version of importing panel stress file
paneldf = pd.read_csv(f'../data/{name}/panel.csv')
paneldf = paneldf.rename(columns={'Elements':'Element ID', 'XX':'sigmaXX', 'YY':'sigmaYY', 'XY':'sigmaXY', 'Loadcase':'Load Case'})
paneldf['Component Name'] = paneldf.apply(elementComponentMatch, axis=1)
paneldf = paneldf.drop(['FileID', 'Step', 'Layer'], axis=1)
paneldf.head(5)


,Element ID,Load Case,sigmaXX,sigmaXY,sigmaYY,Component Name
0,1,1,-41.512531,15.498541,3.740432,panel1
1,2,1,-50.177456,11.455106,6.353047,panel1
2,3,1,-41.343170,13.534094,14.199986,panel1
3,4,1,-41.855709,14.132614,2.756897,panel1
4,5,1,-38.637676,12.126407,6.735441,panel1


# Split up the loadcases 

In [8]:
loadCase1df = paneldf[paneldf['Load Case'] == 1]
loadCase2df = paneldf[paneldf['Load Case'] == 2]
loadCase3df = paneldf[paneldf['Load Case'] == 3]


#Set index back to Element ID 
loadCase1df = loadCase1df.set_index('Element ID')
loadCase2df = loadCase2df.set_index('Element ID')
loadCase3df = loadCase3df.set_index('Element ID')

#Give some overview 
loadCase1df.head(5)

,Load Case,sigmaXX,sigmaXY,sigmaYY,Component Name
Element ID,,,,,
1,1,-41.512531,15.498541,3.740432,panel1
2,1,-50.177456,11.455106,6.353047,panel1
3,1,-41.343170,13.534094,14.199986,panel1
4,1,-41.855709,14.132614,2.756897,panel1
5,1,-38.637676,12.126407,6.735441,panel1


# Now we need to combine into panels:
-Panel ID (as our ID)
-sigmaXXavg
-sigmaYYavg
-sigmaXYavg
-length
-width
-thickness 


In [9]:
# We can just average the stresses @fabianober Tackle rounding!!!! 
#For load case 1 
panelLC1 = loadCase1df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean',
})
dataframeLength = len(panelLC1)
panelLC1['length'] = [length]*dataframeLength
panelLC1['width'] = [width]*dataframeLength
panelLC1['thickness'] = [panelthickness] * dataframeLength
panelLC1['Load Case'] = ['LC1'] * dataframeLength


#For load case 2 
panelLC2 = loadCase2df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean'
})
dataframeLength = len(panelLC2)
panelLC2['length'] = [length]*dataframeLength
panelLC2['width'] = [width]*dataframeLength
panelLC2['thickness'] = [panelthickness] * dataframeLength
panelLC2['Load Case'] = ['LC2'] * dataframeLength


#For load case 3
panelLC3 = loadCase3df.groupby('Component Name').agg({
    'sigmaXX': 'mean',
    'sigmaYY':'mean',
    'sigmaXY':'mean'
})
dataframeLength = len(panelLC3)
panelLC3['length'] = [length]*dataframeLength
panelLC3['width'] = [width]*dataframeLength
panelLC3['thickness'] = [panelthickness] * dataframeLength
panelLC3['Load Case'] = ['LC3'] * dataframeLength


panelLC1

,sigmaXX,sigmaYY,sigmaXY,length,width,thickness,Load Case
Component Name,,,,,,,
panel1,-42.492093,8.097821,13.461565,750,400,8.832,LC1
panel2,-39.399409,8.097821,13.530055,750,400,8.832,LC1
panel3,-39.498893,8.097826,23.833659,750,400,8.832,LC1
panel4,-39.399408,8.097831,13.530006,750,400,8.832,LC1
panel5,-42.492093,8.097831,13.461538,750,400,8.832,LC1


# Now we can apply the functions to the load cases 

In [10]:
panelLC1[['sig_crit_shear', 'sig_crit_biax', 'Reserve Factor']] = panelLC1.apply(panelBuckApply, D11=D11, D12=D12, D22=D22,
                                                                                  D66=D66, axis=1, result_type='expand')
panelLC2[['sig_crit_shear', 'sig_crit_biax', 'Reserve Factor']] = panelLC2.apply(panelBuckApply, D11=D11, D12=D12, D22=D22,
                                                                                  D66=D66, axis=1, result_type='expand')
panelLC3[['sig_crit_shear', 'sig_crit_biax', 'Reserve Factor']] = panelLC3.apply(panelBuckApply, D11=D11, D12=D12, D22=D22,
                                                                                  D66=D66, axis=1, result_type='expand')
panelLC1

,sigmaXX,sigmaYY,sigmaXY,length,width,thickness,Load Case,sig_crit_shear,sig_crit_biax,Reserve Factor
Component Name,,,,,,,,,,
panel1,-42.492093,8.097821,13.461565,750,400,8.832,LC1,108.22899,92.916997,1.387391
panel2,-39.399409,8.097821,13.530055,750,400,8.832,LC1,108.22899,94.407973,1.512491
panel3,-39.498893,8.097826,23.833659,750,400,8.832,LC1,108.22899,94.355591,1.356780
panel4,-39.399408,8.097831,13.530006,750,400,8.832,LC1,108.22899,94.407999,1.512492
panel5,-42.492093,8.097831,13.461538,750,400,8.832,LC1,108.22899,92.917020,1.387392


# Create original output 

In [11]:
#Drop irrelevant columns
panelLC1 = panelLC1.drop(['length', 'width', 'thickness', 'Load Case'], axis=1)
panelLC2 = panelLC2.drop(['length', 'width',  'thickness', 'Load Case'], axis=1)
panelLC3 = panelLC3.drop(['length', 'width',  'thickness', 'Load Case'], axis=1)
panelLC1.head(3)

,sigmaXX,sigmaYY,sigmaXY,sig_crit_shear,sig_crit_biax,Reserve Factor
Component Name,,,,,,
panel1,-42.492093,8.097821,13.461565,108.22899,92.916997,1.387391
panel2,-39.399409,8.097821,13.530055,108.22899,94.407973,1.512491
panel3,-39.498893,8.097826,23.833659,108.22899,94.355591,1.356780


In [12]:
# Rename columns to be unambigous
panelLC1 = panelLC1.rename(columns={"Reserve Factor": "LC 1 RF", 'sigmaXX':'sigmaXXLC1', 'sigmaYY':'sigmaYYLC1', 'sigmaXY':'sigmaXYLC1', 'sig_crit_shear':'sig_crit_shearLC1', 'sig_crit_biax':'sig_crit_biaxLC1'})
panelLC2 = panelLC2.rename(columns={"Reserve Factor": "LC 2 RF", 'sigmaXX':'sigmaXXLC2', 'sigmaYY':'sigmaYYLC2', 'sigmaXY':'sigmaXYLC2', 'sig_crit_shear':'sig_crit_shearLC2', 'sig_crit_biax':'sig_crit_biaxLC2'})
panelLC3 = panelLC3.rename(columns={"Reserve Factor": "LC 3 RF", 'sigmaXX':'sigmaXXLC3', 'sigmaYY':'sigmaYYLC3', 'sigmaXY':'sigmaXYLC3', 'sig_crit_shear':'sig_crit_shearLC3', 'sig_crit_biax':'sig_crit_biaxLC3'})
panelLC1

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,sig_crit_shearLC1,sig_crit_biaxLC1,LC 1 RF
Component Name,,,,,,
panel1,-42.492093,8.097821,13.461565,108.22899,92.916997,1.387391
panel2,-39.399409,8.097821,13.530055,108.22899,94.407973,1.512491
panel3,-39.498893,8.097826,23.833659,108.22899,94.355591,1.356780
panel4,-39.399408,8.097831,13.530006,108.22899,94.407999,1.512492
panel5,-42.492093,8.097831,13.461538,108.22899,92.917020,1.387392


In [13]:
outputDf = pd.concat([panelLC1, panelLC2, panelLC3], axis=1)
outputDf

,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,sig_crit_shearLC1,sig_crit_biaxLC1,LC 1 RF,sigmaXXLC2,sigmaYYLC2,sigmaXYLC2,sig_crit_shearLC2,sig_crit_biaxLC2,LC 2 RF,sigmaXXLC3,sigmaYYLC3,sigmaXYLC3,sig_crit_shearLC3,sig_crit_biaxLC3,LC 3 RF
Component Name,,,,,,,,,,,,,,,,,,
panel1,-42.492093,8.097821,13.461565,108.22899,92.916997,1.387391,-42.823925,-8.097831,13.461972,108.22899,60.042259,0.905260,-13.999976,2.690197,53.633558,108.22899,93.072727,1.285057
panel2,-39.399409,8.097821,13.530055,108.22899,94.407973,1.512491,-39.403190,-8.097831,13.530496,108.22899,58.030529,0.949057,-13.009730,2.690197,53.871393,108.22899,94.534928,1.309098
panel3,-39.498893,8.097826,23.833659,108.22899,94.355591,1.356780,-39.805370,-8.097826,23.833728,108.22899,58.277522,0.882098,-13.089871,2.690217,94.944389,108.22899,94.406705,0.515590
panel4,-39.399408,8.097831,13.530006,108.22899,94.407999,1.512492,-39.403189,-8.097821,13.530447,108.22899,58.030559,0.949057,-13.009728,2.690237,53.871375,108.22899,94.535243,1.309100
panel5,-42.492093,8.097831,13.461538,108.22899,92.917020,1.387392,-42.823926,-8.097821,13.461945,108.22899,60.042289,0.905260,-13.999975,2.690237,53.633549,108.22899,93.073009,1.285058


# ROUND & Output the files

In [14]:
#outputDf = outputDf.round(rounding_digits)
outputDf.to_excel(f'../data/{name}/output/processed_e.xlsx')
outputDf.head(10)


,sigmaXXLC1,sigmaYYLC1,sigmaXYLC1,sig_crit_shearLC1,sig_crit_biaxLC1,LC 1 RF,sigmaXXLC2,sigmaYYLC2,sigmaXYLC2,sig_crit_shearLC2,sig_crit_biaxLC2,LC 2 RF,sigmaXXLC3,sigmaYYLC3,sigmaXYLC3,sig_crit_shearLC3,sig_crit_biaxLC3,LC 3 RF
Component Name,,,,,,,,,,,,,,,,,,
panel1,-42.492093,8.097821,13.461565,108.22899,92.916997,1.387391,-42.823925,-8.097831,13.461972,108.22899,60.042259,0.905260,-13.999976,2.690197,53.633558,108.22899,93.072727,1.285057
panel2,-39.399409,8.097821,13.530055,108.22899,94.407973,1.512491,-39.403190,-8.097831,13.530496,108.22899,58.030529,0.949057,-13.009730,2.690197,53.871393,108.22899,94.534928,1.309098
panel3,-39.498893,8.097826,23.833659,108.22899,94.355591,1.356780,-39.805370,-8.097826,23.833728,108.22899,58.277522,0.882098,-13.089871,2.690217,94.944389,108.22899,94.406705,0.515590
panel4,-39.399408,8.097831,13.530006,108.22899,94.407999,1.512492,-39.403189,-8.097821,13.530447,108.22899,58.030559,0.949057,-13.009728,2.690237,53.871375,108.22899,94.535243,1.309100
panel5,-42.492093,8.097831,13.461538,108.22899,92.917020,1.387392,-42.823926,-8.097821,13.461945,108.22899,60.042289,0.905260,-13.999975,2.690237,53.633549,108.22899,93.073009,1.285058
